## Scraping IMO Board

In [1]:
# Import library needed
import os
import json
import pickle
import ctypes
import pygame
import datetime
import requests
import webbrowser
import numpy as np
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from selenium.webdriver.firefox.options import Options
root_url = "https://www.imo-official.org/problems.aspx"
opts = Options().add_argument("--private")

# Configuration
os.environ['PATH'] += r";C:/SeleniumDrivers"
firefox_driver_path = "C:/SeleniumDrivers/geckodriver.exe"
driver = webdriver.Firefox(executable_path=firefox_driver_path, options=opts)
wait = WebDriverWait(driver, 20)
driver.maximize_window()
driver.get(root_url)
sleep(2)


pygame 2.5.0 (SDL 2.28.0, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [46]:
# Initialize how the data stored
data = {
    'id_key': [],
    'year': [],
    'country': [],
    'all': [],
    'male': [],
    'female': [],
    'p1': [],
    'p2': [],
    'p3': [],
    'p4': [],
    'p5': [],
    'p6': [],
    'p7': [],
    'total': []
}


In [47]:
# The IMO start from 1959 - 2023
years = [str(i) for i in range(1959, 1964+1)]
target_url = "https://www.imo-official.org/year_country_r.aspx?year="

print(f"example url → {target_url+years[0]}")

example url → https://www.imo-official.org/year_country_r.aspx?year=1959


In [48]:
id_key = 1
for year in ["1959", "1960", "1961", "1962"]:
    driver.get(target_url + year)
    
    # Wait for the table to be visible
    table = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'table'))
    )

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')

    for row in table.find_all('tr'):
        cells = row.find_all('td')
        column = len(cells)

        if len(cells) != 0:
            # P1-P6
            if column == 17:
                print("P6")
                country = str(cells[0].text.strip())
                all = (-999 if cells[1].text.strip() == "" else int(cells[1].text.strip()))
                male = (-999 if cells[2].text.strip() == "" else int(cells[2].text.strip()))
                female = (-999 if cells[3].text.strip() == "" else int(cells[3].text.strip()))

                p1 = (-999 if cells[4].text.strip() == "" else int(cells[4].text.strip()))
                p2 = (-999 if cells[5].text.strip() == "" else int(cells[5].text.strip()))
                p3 = (-999 if cells[6].text.strip() == "" else int(cells[6].text.strip()))
                p4 = (-999 if cells[7].text.strip() == "" else int(cells[7].text.strip()))
                p5 = (-999 if cells[8].text.strip() == "" else int(cells[8].text.strip()))
                p6 = (-999 if cells[9].text.strip() == "" else int(cells[9].text.strip()))
                p7 = -999

                total = (-999 if cells[10].text.strip() == "" else int(cells[10].text.strip()))
                
            # P1-P7
            elif column == 18:
                print("P7")
                country = str(cells[0].text.strip())
                all = (-999 if cells[1].text.strip() == "" else int(cells[1].text.strip()))
                male = (-999 if cells[2].text.strip() == "" else int(cells[2].text.strip()))
                female = (-999 if cells[3].text.strip() == "" else int(cells[3].text.strip()))

                p1 = (-999 if cells[4].text.strip() == "" else int(cells[4].text.strip()))
                p2 = (-999 if cells[5].text.strip() == "" else int(cells[5].text.strip()))
                p3 = (-999 if cells[6].text.strip() == "" else int(cells[6].text.strip()))
                p4 = (-999 if cells[7].text.strip() == "" else int(cells[7].text.strip()))
                p5 = (-999 if cells[8].text.strip() == "" else int(cells[8].text.strip()))
                p6 = (-999 if cells[9].text.strip() == "" else int(cells[9].text.strip()))
                p7 = (-999 if cells[10].text.strip() == "" else int(cells[10].text.strip()))

                total = (-999 if cells[11].text.strip() == "" else int(cells[11].text.strip()))
                
            # Store data
            data['id_key'].append(id_key)
            data['year'].append(year)
            
            data['country'].append(country)
            data['all'].append(all)
            data['male'].append(male)
            data['female'].append(female)
            
            data['p1'].append(p1)
            data['p2'].append(p2)
            data['p3'].append(p3)
            data['p4'].append(p4)
            data['p5'].append(p5)
            data['p6'].append(p6)
            data['p7'].append(p7)
            
            data['total'].append(total)
            id_key += 1

    display(pd.DataFrame(data))
    sleep(2)
    # break


P6
P6
P6
P6
P6
P6
P6


,id_key,year,country,all,male,female,p1,p2,p3,p4,p5,p6,p7,total
0,1,1959,Romania,8,6,-999,-999,-999,-999,-999,-999,-999,-999,249
1,2,1959,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,233
2,3,1959,Czechoslovakia,8,5,-999,-999,-999,-999,-999,-999,-999,-999,192
3,4,1959,Bulgaria,8,6,2,-999,-999,-999,-999,-999,-999,-999,131
4,5,1959,Poland,8,7,1,-999,-999,-999,-999,-999,-999,-999,122
5,6,1959,Union of Soviet Socialist Republics,4,4,-999,-999,-999,-999,-999,-999,-999,-999,111
6,7,1959,German Democratic Republic,8,5,3,-999,-999,-999,-999,-999,-999,-999,40


P7
P7
P7
P7
P7


,id_key,year,country,all,male,female,p1,p2,p3,p4,p5,p6,p7,total
0,1,1959,Romania,8,6,-999,-999,-999,-999,-999,-999,-999,-999,249
1,2,1959,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,233
2,3,1959,Czechoslovakia,8,5,-999,-999,-999,-999,-999,-999,-999,-999,192
3,4,1959,Bulgaria,8,6,2,-999,-999,-999,-999,-999,-999,-999,131
4,5,1959,Poland,8,7,1,-999,-999,-999,-999,-999,-999,-999,122
5,6,1959,Union of Soviet Socialist Republics,4,4,-999,-999,-999,-999,-999,-999,-999,-999,111
6,7,1959,German Democratic Republic,8,5,3,-999,-999,-999,-999,-999,-999,-999,40
7,8,1960,Czechoslovakia,8,6,-999,-999,-999,-999,-999,-999,-999,-999,257
8,9,1960,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,248
9,10,1960,Romania,8,4,-999,-999,-999,-999,-999,-999,-999,-999,248


P6
P6
P6
P6
P6
P6


,id_key,year,country,all,male,female,p1,p2,p3,p4,p5,p6,p7,total
0,1,1959,Romania,8,6,-999,-999,-999,-999,-999,-999,-999,-999,249
1,2,1959,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,233
2,3,1959,Czechoslovakia,8,5,-999,-999,-999,-999,-999,-999,-999,-999,192
3,4,1959,Bulgaria,8,6,2,-999,-999,-999,-999,-999,-999,-999,131
4,5,1959,Poland,8,7,1,-999,-999,-999,-999,-999,-999,-999,122
5,6,1959,Union of Soviet Socialist Republics,4,4,-999,-999,-999,-999,-999,-999,-999,-999,111
6,7,1959,German Democratic Republic,8,5,3,-999,-999,-999,-999,-999,-999,-999,40
7,8,1960,Czechoslovakia,8,6,-999,-999,-999,-999,-999,-999,-999,-999,257
8,9,1960,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,248
9,10,1960,Romania,8,4,-999,-999,-999,-999,-999,-999,-999,-999,248


P7
P7
P7
P7
P7
P7
P7


,id_key,year,country,all,male,female,p1,p2,p3,p4,p5,p6,p7,total
0,1,1959,Romania,8,6,-999,-999,-999,-999,-999,-999,-999,-999,249
1,2,1959,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,233
2,3,1959,Czechoslovakia,8,5,-999,-999,-999,-999,-999,-999,-999,-999,192
3,4,1959,Bulgaria,8,6,2,-999,-999,-999,-999,-999,-999,-999,131
4,5,1959,Poland,8,7,1,-999,-999,-999,-999,-999,-999,-999,122
5,6,1959,Union of Soviet Socialist Republics,4,4,-999,-999,-999,-999,-999,-999,-999,-999,111
6,7,1959,German Democratic Republic,8,5,3,-999,-999,-999,-999,-999,-999,-999,40
7,8,1960,Czechoslovakia,8,6,-999,-999,-999,-999,-999,-999,-999,-999,257
8,9,1960,Hungary,8,8,-999,-999,-999,-999,-999,-999,-999,-999,248
9,10,1960,Romania,8,4,-999,-999,-999,-999,-999,-999,-999,-999,248
